In [1]:
import requests
import urllib.request
from urllib.parse import quote
import numpy as np
from tqdm import tqdm
#import contextlibr
import collections
import os
import json
import tqdm
from bs4 import BeautifulSoup as bs
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import collections
import pandas as pd

### Set up spotify scrapping functions

In [2]:
token = 'BQAU7cwi4gE4azyohgwWGhdi9Q6azGYvEwdnM0Iy73qCaHl27LLiwNzSX11f0wOZUrzOxqI1duCcHVbnvKdYLsBVth-IYCATaqtWWrQex2o4bK7zwiEXHWGqWYUh6qRPIox0vKp2uiO8j_P9lWuwGxq25Bw'

header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [3]:
final_json = []
intermediary_results = []

In [4]:
def song_features(song_id):
    query = 'audio-features/' + str(song_id)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()


def get_rateYourMusic_tuples(filenames, table_index=3) :
    
    #song year, artist and title
    searches = []
    
    for f in filenames :

        res = bs(open(f),"html5lib")
        for t in res.findAll(attrs={'class': "mbgen"})[table_index].findAll("tr")[1:] :
                        
            if len(t.findAll("td")) >= 3 :
                
                
            
                year = t.findAll("td")[0].text
                artist = t.findAll("td")[1].text
                title = t.findAll("td")[2].text

                if '/' in title :
                    searches.append([year, artist, title.split('/')[0].strip()])
                    searches.append([year, artist, title.split('/')[1].strip()])

                else :
                    searches.append([year, artist, title])

    return searches

def search_tuples(genre, searches) :
    
    found = 0
    all_info = []

    for e, s in tqdm.tqdm(enumerate(searches)) :
        result  = requests.request('get', 'https://api.spotify.com/v1/search?q=album:'+ quote(s[2]) + "%20artist:" + quote(s[1]) + "&type=track", headers=header).json()
        time.sleep(0.3)
        items = result['tracks']['items']
        
        
        #We found the song
        if len(items) != 0 and ('error' not in result.keys()) :           
            
            album_type = result['tracks']['items'][0]['album']['album_type']
            
            #the song does not come from a compilation
            if album_type != 'compilation' :
            
                album = result['tracks']['items'][0]['album']['name']
                name = result['tracks']['items'][0]['name']

                song_id = result['tracks']['items'][0]['id']   
                song_release = items[0]['album']['release_date']
                features = song_features(song_id)

                if 'error' not in features.keys() :

                    found += 1

                    #get the preview url
                    result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
                    time.sleep(0.3)
                    preview = result_preview['preview_url']

                    if preview is None :
                        preview = ""

                    
                    cover = ""

                    if len(items[0]['album']['images']) != 0 :                
                        cover = items[0]['album']['images'][0]['url']            
                    else :
                        cover = "https://vignette.wikia.nocookie.net/uncyclopedia/images/a/a0/Questionmark.jpg/revision/latest?cb=20090616065802&format=original"

                    artists = s[1]



                    song_dict = {'year' : song_release,
                                 'artists' : artists,
                                 'name' : name,
                                 'cover' : cover,
                                 'album' : album,
                                 'genre' : genre,
                                'danceability' : features['danceability'],
                                'energy' : features['energy'],
                                'key' : features['key'],
                                'loudness' : features['loudness'],
                                'mode' : features['mode'],
                                'speechiness' : features['speechiness'],
                                'acousticness' : features['acousticness'],
                                'instrumentalness' : features['instrumentalness'],
                                'liveness' : features['liveness'],
                                'valence' : features['valence'],
                                'tempo' : features['tempo'],
                                'duration_ms' : features['duration_ms'],
                                'preview' : preview}


                    all_info.append(song_dict)
                    
            
                
    print("found : ", found , ", (" + str(100*found/len(searches)) + "%) for genre " + genre)
    return all_info


# RateYourMusic

### Download Contemporary RnB titles (93 songs)

From Itunes (70)

In [5]:
# get names from itunes
url = 'https://itunes.apple.com/search?term=song&genreId=1136&limit=200'

response = requests.get(url)
data = response.json()

found = 0

for e in tqdm.tqdm(data['results']):
    result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(e['trackName']) + "%20artist:" + quote(e['artistName']) + "&type=track", headers=header).json()
    
    items = result['tracks']['items']
    
    
    #We found the song
    if len(items) != 0 and ('error' not in result.keys()):
        
        found += 1
    
        song_id = result['tracks']['items'][0]['id']    
        features = song_features(song_id)
        
        #get the preview url
        result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
        time.sleep(0.3)
        preview = result_preview['preview_url']

        if preview is None :
            preview = ""

        year = e['releaseDate']
        cover = e['artworkUrl100']
        artists = " and ".join([a['name'] for a in result['tracks']['items'][0]['artists']])
        name = e['trackName']
        album = result['tracks']['items'][0]['album']['name']



        song_dict = {'year' : year,
                     'artists' : artists,
                     'name' : name,
                     'cover' : cover,
                     'album' : album,
                     'genre' : "contemporary-rnb",
                    'danceability' : features['danceability'],
                    'energy' : features['energy'],
                    'key' : features['key'],
                    'loudness' : features['loudness'],
                    'mode' : features['mode'],
                    'speechiness' : features['speechiness'],
                    'acousticness' : features['acousticness'],
                    'instrumentalness' : features['instrumentalness'],
                    'liveness' : features['liveness'],
                    'valence' : features['liveness'],
                    'tempo' : features['tempo'],
                    'duration_ms' : features['duration_ms'],
                    'preview' : preview}


        final_json.append(song_dict)
        
intermediary_results = final_json.copy()

  0%|          | 0/94 [00:00<?, ?it/s]


KeyError: 'tracks'

### Download british RnB (148 songs)

In [ ]:
file_names_brit = []
for nb in range(250) :
    file_name = "data/british_rnb/british_rnb_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_brit.append(file_name)

#break the search in two
brit_tuples = get_rateYourMusic_tuples(file_names_brit)    


final_json.extend(search_tuples("british-rnb", brit_tuples[:int(len(brit_tuples)/2)]))
intermediary_results = final_json.copy()

final_json.extend(search_tuples("british-rnb", brit_tuples[int(len(brit_tuples)/2):]))
intermediary_results = final_json.copy()

### Download alternative RnB (640 songs)

In [ ]:
file_names_alt = []
for nb in range(250) :
    file_name = "data/alternative_rnb/alt_songs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_alt.append(file_name)

alt_tuples = get_rateYourMusic_tuples(file_names_alt)

        
final_json.extend(search_tuples("alternative-rnb", alt_tuples))
intermediary_results = final_json.copy()

### Download new jack swing RnB (146 songs)

In [ ]:
file_names_nj = []
for nb in range(250) :
    file_name = "data/new_jack_swing_rnb/njs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_nj.append(file_name)

njs_tuples = get_rateYourMusic_tuples(file_names_nj)
        
final_json.extend(search_tuples("new-jack-swing-rnb", njs_tuples))
intermediary_results = final_json.copy()

### Download new Orleans RnB (247 songs)

In [ ]:
file_names_no = []
for nb in range(250) :
    file_name = "data/new_orleans_rnb/no_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_no.append(file_name)
        
no_tuples = get_rateYourMusic_tuples(file_names_no)

final_json.extend(search_tuples("new-orleans-rnb", no_tuples))
intermediary_results = final_json.copy()

### Download Jump Blues (159 songs)

In [ ]:
file_names_jb = []
for nb in range(250) :
    file_name = "data/jump_blues_rnb/jp_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_jb.append(file_name)

jb_tuples = get_rateYourMusic_tuples(file_names_jb)
        
final_json.extend(search_tuples("jump-blues", jb_tuples))
intermediary_results = final_json.copy()

### Download blue-eyed soul (622 songs)

In [ ]:
file_names_bes = []
for nb in range(250) :
    file_name = "data/blue_eyed_soul/blue_eyed_soul_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_bes.append(file_name)

bes_tuples = get_rateYourMusic_tuples(file_names_bes)
        
final_json.extend(search_tuples("blue-eyed-soul", bes_tuples))



0it [00:00, ?it/s]

1it [00:00,  1.06it/s]

2it [00:01,  1.28it/s]

3it [00:01,  1.46it/s]

4it [00:02,  1.60it/s]

5it [00:03,  1.40it/s]

6it [00:03,  1.60it/s]

7it [00:04,  1.77it/s]

8it [00:04,  1.90it/s]

9it [00:04,  2.02it/s]

10it [00:05,  1.59it/s]

11it [00:06,  1.76it/s]

12it [00:07,  1.48it/s]

13it [00:08,  1.32it/s]

14it [00:08,  1.51it/s]

15it [00:09,  1.70it/s]

16it [00:09,  1.82it/s]

17it [00:10,  1.50it/s]

18it [00:11,  1.30it/s]

19it [00:11,  1.51it/s]

20it [00:12,  1.66it/s]

21it [00:12,  1.84it/s]

22it [00:13,  1.49it/s]

23it [00:14,  1.35it/s]

24it [00:15,  1.54it/s]

25it [00:15,  1.74it/s]

26it [00:16,  1.50it/s]

27it [00:16,  1.64it/s]

28it [00:17,  1.80it/s]

29it [00:17,  1.97it/s]

30it [00:18,  2.10it/s]

31it [00:18,  2.19it/s]

32it [00:18,  2.20it/s]

33it [00:19,  2.27it/s]

34it [00:20,  1.76it/s]

35it [00:21,  1.51it/s]

36it [00:21,  1.73it/s]

37it [00:22,  1.47it/s]

38it [00:22,  1.67it/s]

39it [00:23,  1.84it/s]

40it [00:23,

515it [04:26,  2.23it/s]

516it [04:26,  2.28it/s]

517it [04:27,  1.74it/s]

518it [04:28,  1.49it/s]

519it [04:29,  1.49it/s]

520it [04:29,  1.53it/s]

521it [04:30,  1.74it/s]

522it [04:30,  1.86it/s]

523it [04:31,  1.56it/s]

524it [04:31,  1.74it/s]

525it [04:32,  1.88it/s]

526it [04:33,  1.56it/s]

527it [04:33,  1.72it/s]

528it [04:34,  1.87it/s]

529it [04:34,  2.03it/s]

530it [04:34,  2.14it/s]

531it [04:35,  2.23it/s]

532it [04:35,  2.23it/s]

533it [04:36,  1.76it/s]

534it [04:36,  1.89it/s]

535it [04:37,  2.02it/s]

536it [04:37,  2.13it/s]

537it [04:38,  2.20it/s]

538it [04:38,  2.28it/s]

539it [04:39,  1.76it/s]

540it [04:39,  1.89it/s]

541it [04:40,  1.56it/s]

542it [04:41,  1.73it/s]

543it [04:41,  1.88it/s]

544it [04:42,  1.53it/s]

545it [04:43,  1.36it/s]

546it [04:43,  1.57it/s]

547it [04:44,  1.76it/s]

548it [04:44,  1.90it/s]

549it [04:45,  2.01it/s]

550it [04:46,  1.63it/s]

551it [04:47,  1.43it/s]

552it [04:47,  1.64it/s]

553it [04:47

1026it [09:11,  2.12it/s]

1027it [09:12,  2.17it/s]

1028it [09:12,  2.22it/s]

1029it [09:13,  2.26it/s]

1030it [09:13,  2.32it/s]

1031it [09:14,  2.01it/s]

1032it [09:14,  1.92it/s]

1033it [09:15,  1.80it/s]

1034it [09:15,  1.82it/s]

1035it [09:17,  1.36it/s]

1036it [09:18,  1.25it/s]

1037it [09:18,  1.46it/s]

1038it [09:19,  1.51it/s]

1039it [09:19,  1.36it/s]

1040it [09:20,  1.57it/s]

1041it [09:20,  1.75it/s]

1042it [09:21,  1.92it/s]

1043it [09:21,  2.02it/s]

1044it [09:22,  2.11it/s]

1045it [09:22,  2.21it/s]

1046it [09:22,  2.29it/s]

1047it [09:23,  2.35it/s]

1048it [09:23,  2.35it/s]

1049it [09:24,  2.40it/s]

1050it [09:24,  2.42it/s]

1051it [09:25,  1.55it/s]

1052it [09:26,  1.73it/s]

1053it [09:26,  1.87it/s]

1054it [09:26,  2.04it/s]

1055it [09:27,  2.16it/s]

1056it [09:27,  2.24it/s]

1057it [09:28,  2.26it/s]

1058it [09:28,  2.30it/s]

1059it [09:28,  2.35it/s]

1060it [09:29,  2.39it/s]

1061it [09:29,  2.37it/s]

1062it [09:30,  2.36it/s]

1

1522it [13:31,  1.49it/s]

1523it [13:32,  1.67it/s]

1524it [13:32,  1.85it/s]

1525it [13:33,  2.02it/s]

1526it [13:33,  2.16it/s]

1527it [13:33,  2.22it/s]

1528it [13:34,  2.31it/s]

1529it [13:34,  2.36it/s]

1530it [13:35,  1.74it/s]

1531it [13:36,  1.90it/s]

1532it [13:37,  1.57it/s]

1533it [13:37,  1.76it/s]

1534it [13:38,  1.51it/s]

1535it [13:39,  1.39it/s]

1536it [13:39,  1.56it/s]

1537it [13:40,  1.73it/s]

1538it [13:40,  1.91it/s]

1539it [13:41,  1.58it/s]

1540it [13:41,  1.75it/s]

1541it [13:42,  1.49it/s]

1542it [13:43,  1.62it/s]

1543it [13:43,  1.79it/s]

1544it [13:43,  1.95it/s]

1545it [13:44,  2.08it/s]

1546it [13:45,  1.65it/s]

1547it [13:45,  1.81it/s]

1548it [13:46,  1.97it/s]

1549it [13:47,  1.59it/s]

1550it [13:47,  1.38it/s]

1551it [13:48,  1.58it/s]

1552it [13:49,  1.41it/s]

1553it [13:49,  1.61it/s]

1554it [13:50,  1.77it/s]

1555it [13:50,  1.96it/s]

1556it [13:50,  2.10it/s]

1557it [13:51,  2.19it/s]

1558it [13:51,  2.21it/s]

1

2018it [18:40,  1.19it/s]

2019it [18:40,  1.16it/s]

2020it [18:42,  1.04it/s]

2021it [18:42,  1.25it/s]

2022it [18:43,  1.17it/s]

2023it [18:43,  1.37it/s]

2024it [18:44,  1.52it/s]

2025it [18:45,  1.34it/s]

2026it [18:46,  1.23it/s]

2027it [18:46,  1.44it/s]

2028it [18:47,  1.62it/s]

2029it [18:47,  1.82it/s]

2030it [18:48,  1.94it/s]

2031it [18:48,  2.03it/s]

2032it [18:49,  1.59it/s]

2033it [18:49,  1.76it/s]

2034it [18:50,  1.46it/s]

2035it [18:51,  1.32it/s]

2036it [18:52,  1.54it/s]

2037it [18:52,  1.73it/s]

2038it [18:52,  1.87it/s]

2039it [18:53,  1.96it/s]

2040it [18:54,  1.33it/s]

2041it [18:56,  1.07it/s]

2042it [18:57,  1.05it/s]

2043it [18:57,  1.17it/s]

2044it [18:58,  1.17it/s]

2045it [18:59,  1.37it/s]

2046it [18:59,  1.55it/s]

2047it [19:00,  1.39it/s]

2048it [19:01,  1.30it/s]

2049it [19:02,  1.26it/s]

2050it [19:02,  1.49it/s]

2051it [19:02,  1.69it/s]

2052it [19:03,  1.87it/s]

2053it [19:03,  1.99it/s]

2054it [19:04,  2.11it/s]

2

found :  622 , (27.817531305903398%) for genre blue-eyed-soul


In [ ]:
#Dump everything into the data
json.dump(final_json, open("rnb_data.json", 'w'))

# AllMusic

### Contemporary rnb (720)

In [ ]:
cont_tuples = json.load(open("data/All_Music/parsed/am_contemporary_rnb.json"))
final_json.extend(search_tuples("contemporary-rnb", cont_tuples))
intermediary_results = final_json.copy()



0it [00:00, ?it/s]

1it [00:00,  1.00it/s]

2it [00:02,  1.02s/it]

3it [00:02,  1.02it/s]

4it [00:03,  1.06it/s]

5it [00:04,  1.09it/s]

6it [00:05,  1.03it/s]

7it [00:06,  1.00s/it]

8it [00:07,  1.18it/s]

9it [00:08,  1.17it/s]

10it [00:09,  1.16it/s]

11it [00:09,  1.16it/s]

12it [00:11,  1.07it/s]

13it [00:11,  1.08it/s]

14it [00:12,  1.05it/s]

15it [00:13,  1.03it/s]

16it [00:14,  1.06it/s]

17it [00:15,  1.08it/s]

18it [00:16,  1.11it/s]

19it [00:16,  1.32it/s]

20it [00:17,  1.53it/s]

21it [00:17,  1.70it/s]

22it [00:18,  1.50it/s]

23it [00:19,  1.38it/s]

24it [00:20,  1.30it/s]

25it [00:20,  1.49it/s]

26it [00:21,  1.69it/s]

27it [00:22,  1.48it/s]

28it [00:23,  1.23it/s]

29it [00:24,  1.10it/s]

30it [00:25,  1.10it/s]

31it [00:26,  1.11it/s]

32it [00:27,  1.12it/s]

33it [00:27,  1.11it/s]

34it [00:28,  1.12it/s]

35it [00:29,  1.11it/s]

36it [00:30,  1.12it/s]

37it [00:31,  1.07it/s]

38it [00:32,  1.01it/s]

39it [00:33,  1.01s/it]

40it [00:34,

515it [06:47,  1.39it/s]

516it [06:48,  1.61it/s]

517it [06:48,  1.81it/s]

518it [06:49,  1.53it/s]

519it [06:50,  1.40it/s]

520it [06:51,  1.30it/s]

521it [06:51,  1.52it/s]

522it [06:52,  1.37it/s]

523it [06:53,  1.28it/s]

524it [06:53,  1.51it/s]

525it [06:54,  1.40it/s]

526it [06:54,  1.62it/s]

527it [06:55,  1.76it/s]

528it [06:56,  1.50it/s]

529it [06:57,  1.37it/s]

530it [06:58,  1.30it/s]

531it [06:58,  1.53it/s]

532it [06:58,  1.69it/s]

533it [06:59,  1.88it/s]

534it [06:59,  2.01it/s]

535it [07:00,  2.10it/s]

536it [07:00,  2.19it/s]

537it [07:00,  2.28it/s]

538it [07:01,  1.72it/s]

539it [07:02,  1.87it/s]

540it [07:02,  2.01it/s]

541it [07:03,  2.06it/s]

542it [07:03,  1.66it/s]

543it [07:04,  1.47it/s]

544it [07:05,  1.64it/s]

545it [07:05,  1.81it/s]

546it [07:06,  1.98it/s]

547it [07:06,  2.06it/s]

548it [07:06,  2.16it/s]

549it [07:07,  2.19it/s]

550it [07:07,  2.23it/s]

551it [07:08,  2.28it/s]

552it [07:09,  1.75it/s]

553it [07:09

1026it [12:08,  1.62it/s]

1027it [12:09,  1.44it/s]

1028it [12:10,  1.33it/s]

1029it [12:10,  1.55it/s]

1030it [12:11,  1.39it/s]

1031it [12:12,  1.33it/s]

1032it [12:13,  1.28it/s]

1033it [12:13,  1.50it/s]

1034it [12:14,  1.70it/s]

1035it [12:15,  1.49it/s]

1036it [12:16,  1.41it/s]

1037it [12:16,  1.63it/s]

1038it [12:17,  1.30it/s]

1039it [12:18,  1.21it/s]

1040it [12:19,  1.19it/s]

1041it [12:20,  1.19it/s]

1042it [12:20,  1.40it/s]

1043it [12:21,  1.61it/s]

1044it [12:21,  1.79it/s]

1045it [12:21,  1.91it/s]

1046it [12:22,  2.09it/s]

1047it [12:22,  2.19it/s]

1048it [12:23,  2.28it/s]

1049it [12:23,  2.35it/s]

1050it [12:23,  2.38it/s]

1051it [12:24,  1.80it/s]

1052it [12:25,  1.53it/s]

1053it [12:26,  1.75it/s]

1054it [12:26,  1.90it/s]

1055it [12:27,  1.59it/s]

1056it [12:28,  1.44it/s]

1057it [12:29,  1.33it/s]

1058it [12:29,  1.56it/s]

1059it [12:30,  1.39it/s]

1060it [12:31,  1.29it/s]

1061it [12:31,  1.51it/s]

1062it [12:32,  1.70it/s]

1

found :  720 , (55.172413793103445%) for genre contemporary-rnb


### early rnb (198)

In [ ]:
early_tuples = json.load(open("data/All_Music/parsed/am_early_rnb.json"))
final_json.extend(search_tuples("early-rnb", early_tuples))
intermediary_results = final_json.copy()



0it [00:00, ?it/s]

1it [00:00,  1.13it/s]

2it [00:01,  1.12it/s]

3it [00:02,  1.34it/s]

4it [00:03,  1.27it/s]

5it [00:03,  1.48it/s]

6it [00:03,  1.67it/s]

7it [00:04,  1.85it/s]

8it [00:04,  2.00it/s]

9it [00:05,  1.63it/s]

10it [00:06,  1.42it/s]

11it [00:06,  1.63it/s]

12it [00:07,  1.81it/s]

13it [00:08,  1.52it/s]

14it [00:09,  1.35it/s]

15it [00:10,  1.29it/s]

16it [00:10,  1.48it/s]

17it [00:10,  1.66it/s]

18it [00:11,  1.42it/s]

19it [00:12,  1.60it/s]

20it [00:12,  1.80it/s]

21it [00:13,  1.54it/s]

22it [00:13,  1.71it/s]

23it [00:14,  1.86it/s]

24it [00:14,  2.00it/s]

25it [00:15,  2.14it/s]

26it [00:15,  2.19it/s]

27it [00:16,  1.68it/s]

28it [00:16,  1.85it/s]

29it [00:17,  1.54it/s]

30it [00:18,  1.73it/s]

31it [00:18,  1.89it/s]

32it [00:19,  2.02it/s]

33it [00:19,  2.10it/s]

34it [00:20,  1.65it/s]

35it [00:20,  1.78it/s]

36it [00:21,  1.52it/s]

37it [00:22,  1.71it/s]

38it [00:22,  1.88it/s]

39it [00:23,  2.02it/s]

40it [00:23,

515it [04:43,  2.11it/s]

516it [04:44,  2.19it/s]

517it [04:45,  1.70it/s]

518it [04:45,  1.85it/s]

519it [04:46,  1.94it/s]

520it [04:46,  1.56it/s]

521it [04:47,  1.39it/s]

522it [04:48,  1.28it/s]

523it [04:49,  1.49it/s]

524it [04:49,  1.68it/s]

525it [04:50,  1.86it/s]

526it [04:50,  2.00it/s]

527it [04:50,  2.08it/s]

528it [04:51,  1.65it/s]

529it [04:52,  1.84it/s]

530it [04:52,  1.93it/s]

531it [04:53,  2.05it/s]

532it [04:53,  2.12it/s]

533it [04:54,  1.67it/s]

534it [04:54,  1.79it/s]

535it [04:55,  1.93it/s]

536it [04:56,  1.59it/s]

537it [04:56,  1.77it/s]

538it [04:57,  1.51it/s]

539it [04:58,  1.36it/s]

540it [04:58,  1.55it/s]

541it [04:59,  1.73it/s]

542it [04:59,  1.84it/s]

543it [05:00,  1.95it/s]

544it [05:00,  2.07it/s]

545it [05:01,  2.05it/s]

546it [05:01,  2.15it/s]

547it [05:02,  1.66it/s]

548it [05:02,  1.81it/s]

549it [05:03,  1.49it/s]

550it [05:04,  1.68it/s]

551it [05:04,  1.83it/s]

552it [05:05,  1.98it/s]

553it [05:05

found :  198 , (22.398190045248867%) for genre early-rnb


### motown (86)

In [ ]:
mo_tuples = json.load(open("data/All_Music/parsed/am_motown.json"))
final_json.extend(search_tuples("motown-rnb", mo_tuples))
intermediary_results = final_json.copy()



0it [00:00, ?it/s]

1it [00:00,  2.43it/s]

2it [00:01,  1.82it/s]

3it [00:01,  1.87it/s]

4it [00:02,  1.97it/s]

5it [00:03,  1.58it/s]

6it [00:03,  1.75it/s]

7it [00:04,  1.50it/s]

8it [00:05,  1.36it/s]

9it [00:06,  1.28it/s]

10it [00:06,  1.49it/s]

11it [00:07,  1.35it/s]

12it [00:07,  1.56it/s]

13it [00:08,  1.76it/s]

14it [00:08,  1.93it/s]

15it [00:09,  2.05it/s]

16it [00:10,  1.63it/s]

17it [00:10,  1.84it/s]

18it [00:11,  1.80it/s]

19it [00:12,  1.12it/s]

20it [00:13,  1.09it/s]

21it [00:14,  1.10it/s]

22it [00:15,  1.07it/s]

23it [00:16,  1.09it/s]

24it [00:16,  1.27it/s]

25it [00:17,  1.48it/s]

26it [00:18,  1.31it/s]

27it [00:18,  1.49it/s]

28it [00:19,  1.68it/s]

29it [00:19,  1.79it/s]

30it [00:20,  1.53it/s]

31it [00:21,  1.69it/s]

32it [00:21,  1.46it/s]

33it [00:22,  1.32it/s]

34it [00:23,  1.54it/s]

35it [00:23,  1.71it/s]

36it [00:24,  1.89it/s]

37it [00:25,  1.53it/s]

38it [00:25,  1.68it/s]

39it [00:26,  1.43it/s]

40it [00:26,

found :  86 , (24.85549132947977%) for genre motown-rnb


In [ ]:
json.dump(final_json, open("rnb_data.json", 'w'))

# visualize number of songs per year

In [ ]:
import collections

#all possible genres
genres = ['contemporary-rnb', 'british-rnb', 'new-orleans-rnb', 
          'alternative-rnb', 'new-jack-swing-rnb', 'blue-eyed-soul', 'jump-blues']

#compute earliest and latest year globally
all_years = [d['year'] for d in final_json if (d['year'] != '')]
min_all_years, max_all_years = int(min(all_years)), int(max(all_years)) + 1

#plot settings
width = 1
figure(figsize=(15,10))
plots = []

#initialize all years to zero
seen_years = collections.Counter()
for y in range(min_all_years, max_all_years) :
        seen_years[str(y)] = 0



for g in genres :

    #years of genre songs
    genre_years = [d['year'] for d in final_json if ((d["genre"] == g) and (d['year'] != ''))]
    
    #create counter
    years_per_genre = collections.Counter()
    
    
    #put 0 as default value for all years
    for y in range(min_all_years, max_all_years) :
        years_per_genre[str(y)] = 0
        
    #count values
    years_per_genre.update(genre_years)
    
    #create plot variables
    labels, values = zip(*years_per_genre.items())
    labels_all, values_all = zip(*seen_years.items())
    indexes = np.arange(len(labels))    
    
    #plot with bottom
    p = plt.bar(indexes, values, width, bottom=values_all)    
    plots.append(p)
    
    #update seen years counter
    seen_years.update(genre_years)

#settings for axes
plt.xticks(indexes + width * 0.5, labels, rotation=90)
plt.xlabel("release year")
plt.ylabel("number of songs")
plt.ylim(ymax = 160, ymin = 0)

#general plot settings
plt.title("number of RnB songs per genre and per year")
plt.legend(plots, genres)

# plot
plt.savefig("nb_of_songs.png")
plt.show()


# compute discriminative coefficients for each feature and each year 

In [ ]:
[f for f in final_json_bis if f['genre'] == 'contemporary-rnb']

In [ ]:
years = range(1900, 2050)
best_features = dict()

for y in years :
    
    # load data
    names = ['energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    targets = np.array([c['genre'] for c in final_json if c['year'] == str(y)])
    
    if len(set(targets)) > 1:

        features = np.array([c['danceability'] for c in final_json if c['year'] == str(y)])
        
        
        for n in names :
            features = np.vstack((features, np.array([c[n] for c in final_json if c['year'] == str(y)])))


        model = LogisticRegression()
        rfe = RFE(model, 2)
        fit = rfe.fit(features.T, targets)

        best_for_year = sorted([c[0] for c in zip(['danceability'] + names, fit.support_) if c[1]])
        best_features[y] = best_for_year
        
best_features